### Instructions for Running the Code

To reproduce the results and execute the code of this repo successfully, please follow these steps:

1. Ensure that you are connected to a T4 GPU runtime, as our code relies on NVIDIA CUDA for acceleration and may not work efficiently on CPUs.
2. Run the code snippets provided below sequentially.
3. Pay attention to any comments within the code snippets.
4. After running all code snippets, you can see the evaluation metrics and reconstructed images in experiments folder.

Please note: You need to have access to view the [dataset](https://drive.google.com/drive/folders/1mElxNk6-XAfSeJcK76oZuXvU5zAYF7FS?usp=sharing), and since we're running this on Drive, you need to have enough storage space in your Google Drive for this to work.

In [ ]:
# Mounting the Google Drive onto Colab.
# Note: Connect to the T4 GPU runtime as our code requires GPU acceleration due to NVIDIA CUDA dependencies.

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Run the following code only when executing it for the first time to clone the GitHub repository into your google drive.

# UNCOMMENT THE CODE BELOW WHEN RUNNING FOR THE FIRST TIME.
# %%bash
# cd /content/drive/MyDrive/
# mkdir -p manogna-dp-project
# cd manogna-dp-project
# git clone https://github.com/manognachennuru/rog.git


The below coding snippet is to change the current working directory to the home directory

In [ ]:
# Printing the current working directory at the beginning
print("Working Directory at the beginning:", end="")
!pwd

import os

# Define the path to the directory where your data is located
home_directory = "/content/drive/MyDrive/manogna-dp-project/rog"

# Change the current directory to the data directory
os.chdir(home_directory)

# Printing the current working directory after changing
print("Working Directory:", end="")
!pwd


## Install Python Packages

In the following cells, we'll install the required Python packages. Some packages are installed using a `requirements.txt` file, while others will be installed individually. If you encounter any errors during installation, you can safely ignore them unless they prevent the notebook from running successfully.

Please proceed to the next cells for package installation.


In [ ]:
# Install Python packages
# Some packages are installed using requirements.txt. Individual statements are provided for those that are not installed using requirements.
# Ignore the error for opencv_python==4.5.2.52 as it seems there's no matching distribution.

!pip3 install -r requirements.txt


In [ ]:
pip install inplace-abn

In [ ]:
pip install lpips

## Check installations

 The below code is just to see if all the dependencies are installed properly. I put them in try-except blocks so that it won't raise an error. It instead prints a statement if the respective package is not installed.

In [ ]:
try:
    import inplace_abn
    print("inplace_abn version:", inplace_abn.__version__)
except ImportError:
    print("inplace_abn is not installed")

try:
    import lpips
    print("lpips installed")
except ImportError:
    print("lpips is not installed")

try:
    import numpy
    print("numpy version:", numpy.__version__)
except ImportError:
    print("numpy is not installed")

try:
    import cv2
    print("opencv-python version:", cv2.__version__)  # OpenCV is often imported as cv2
except ImportError:
    print("opencv-python is not installed")

try:
    import pandas
    print("pandas version:", pandas.__version__)
except ImportError:
    print("pandas is not installed")

try:
    import PIL
    print("Pillow version:", PIL.__version__)
except ImportError:
    print("Pillow is not installed")

try:
    import yaml
    print("PyYAML version:", yaml.__version__)
except ImportError:
    print("PyYAML is not installed")

try:
    import scipy
    print("scipy version:", scipy.__version__)
except ImportError:
    print("scipy is not installed")

try:
    import torch
    print("PyTorch version:", torch.__version__)
except ImportError:
    print("PyTorch is not installed")

try:
    import torchvision
    print("torchvision version:", torchvision.__version__)
except ImportError:
    print("torchvision is not installed")



## Download Pretrained Models

In this step, we'll download the pretrained models and place them under the `model_zoos` directory. Bash commands will be used to move the files into the appropriate folder.

Please execute the following commands to download and organize the pretrained models.

[link to the pre-trained models](https://huggingface.co/erickyue/rog_modelzoo/tree/main)


In [ ]:
# NOTE: Uncomment the following lines if you're running it for the first time.

#Change directory to the model_zoos folder
import os
os.chdir("/content/drive/MyDrive/manogna-dp-project/rog/model_zoos")

#Print the current directory
!pwd

#Install git-lfs for handling large files
!apt install git-lfs
!git lfs install

#Clone the repository containing pretrained models
!git clone https://huggingface.co/erickyue/rog_modelzoo

#Move the downloaded files to the current directory
!mv -v /content/drive/MyDrive/manogna-dp-project/rog/model_zoos/rog_modelzoo/*  ./

#Remove the empty rog_modelzoo directory
!rm -rf rog_modelzoo/

## Dataset Access

In order to reproduce the results, you need to access the shared dataset. I have shared the folder with Dr. Fan (Course Instructor) and Harshitha (Teaching Assistant). If you're using with different account, Please send me a request.

1. Locate the shared folder link provided to you.
2. Go to your Google Drive and click on "Shared with me" to find the folder.
3. Right-click on the shared folder and select "Add shortcut to Drive."
4. Save the shortcut in the following directory: `/content/drive/MyDrive`.

Once you have added the shortcut, you will be able to access the dataset from within the notebook.



In [ ]:
#NOTE: Here, you add shortcut to the shared folder and save it in /content/drive/MyDrive.

The below code snippet changes the active directory. It is needed to avoid path conflicts later on.

In [ ]:
# Define the path to the directory where your data is located
data_directory = "/content/drive/MyDrive/manogna-dp-project/rog"

# Change the current directory to the data directory
os.chdir(data_directory)

# Print the current working directory after changing
!pwd

## Print Updated Datapair File

I have updated the `datapair.dat` file to match the paths of the current dataset. Please note that the paths in the `datapair.dat` file have been modified to correspond to the paths in the current dataset, ensuring compatibility with the code execution.


In [ ]:
# Code snippet to print some of the contents of datapair.dat

# Define the path to the datapair file
datapair_file_path = "data/datapair.dat"

import os
import pickle

# Load the datapair from the file
with open(datapair_file_path, "rb") as fp:
    record = pickle.load(fp)

# Print the datapair
print("datapair:")
for index, item in enumerate(record["data_pair"]):
    print(f"Index {index}: {item}")
    if index == 10:
      break

# Check if 'root' key exists and print it if it does
if "root" in record:
    print(f"Root directory: {record['root']}")


## Running the Model and Evaluating the Performance

The following snippets run some of the examples. The results will be stored under the `experiments` directory.

Peak Signal-to-Noise Ratio (PSNR), Structural Similarity Index (SSIM), and Learned Perceptual Image Patch Similarity (LPIPS) of the results are evaluated and printed.

Original and reconstructed images (for minimum runnable image set) are also displayed in the experiments folder to visualize the results.


In [ ]:
!python3 main.py

In [ ]:
!python3 attack_fedavg.py